In [26]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import pandas as pd
import plotly.plotly as plotly
from plotly.graph_objs import *

In [2]:
#  Make notebook and pandas DataFrames look better
from IPython.core.display import HTML
css = open("../styles/style-table.css").read() + open('../styles/style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [3]:
from model_def import get_payouts
max_tip = 5
num_tips = 100
curator_perc = 0.4
yours_perc = 0.1
df = get_payouts(max_tip=max_tip, num_tips=num_tips, curator_payout_perc=curator_perc, yours_payout_perc=yours_perc)
df.head()

Total tipped:    $221.35
Yours revenue:   $22.13
Creator revenue: $110.67
Curator revenue: $88.54


,tip,curator_payouts,yours_revenue,curator_num,total_earned
0,2.31685,"[0.926740584306, 0.451587069067, 0.44781392460...",0.231685,0,4.688070
1,2.20194,"[0.429189460687, 0.425603454915, 0.20268443417...",0.220194,1,3.574777
2,4.22524,"[0.816677160526, 0.388924822556, 0.32108592628...",0.422524,2,6.035970
3,2.61363,"[0.240579138754, 0.198615699314, 0.15304725486...",0.261363,3,3.228524
4,2.66381,"[0.20242959196, 0.155986125264, 0.048952237620...",0.266381,4,3.045320


In [42]:
sizemode='area'
sizeref = df['total_earned'].max() / 1e2**1.5
labels = []
for exact in df['total_earned']:
    labels.append('${0:.2f}'.format(round(exact,2)))

# Define a trace-generating function (returns a Scatter object)
data = Data()
data.append(Scatter(
        x=df['curator_num'],              # Curator number
        y=df['tip'],                      # Amount of initial tip
        text=labels,                      # label on hover
        mode='markers',                   # (!) point markers only on this plot
        marker= Marker(       
            size=df.total_earned,         # (!) marker sizes (sizes is a list)
            sizeref=sizeref,              # link sizeref
            sizemode=sizemode,            # link sizemode
            opacity=0.5,                  # (!) partly transparent markers
            line=Line(width=0.0)          # remove marker borders
        )
    ))

# Set plot and axis titles
title = "Projected Yours Network Payout"
x_title = "Curator Number"
y_title = "Amount Tipped, (USD)"

# Define a dictionary of axis style options
axis_style = dict(
    zeroline=False,       # remove thick zero line
    gridcolor='#FFFFFF',  # white grid lines
    ticks='outside',      # draw ticks outside axes 
    ticklen=8,            # tick length
    tickwidth=1.5         #   and width
)

# Make layout object
layout = Layout(
    title=title,             # set plot title
    plot_bgcolor='#EFECEA',  # set plot color to grey
    xaxis=XAxis(
        axis_style,      # add axis style dictionary
        title=x_title,   # x-axis title
    ),
    yaxis=YAxis(
        axis_style,      # add axis style dictionary
        title=y_title,   # y-axis title
    )
)

# Make Figure object
fig = Figure(data=data, layout=layout)

# (@) Send to Plotly and show in notebook
plotly.iplot(fig, filename='paymentModel')
